Import des librairies nécessaires à la préparation des données

In [2]:
import pandas as pd
import os
import glob
from IPython.display import display

Récupération du fichier avec la correspondance id_station et coordonnées GPS pour ne garder que les fichiers des stations près de NYC

Phase 1 : Initialisation de l'Environnement

In [3]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import requests
import json
from math import radians, cos, sin, asin, sqrt

# --- 1. Configuration de Robustesse et Initialisation Spark ---
# Augmentation des timeouts et allocation de mémoire stricte pour éviter les crashs JVM
spark = SparkSession.builder \
    .appName("DataLake_NOAA_NYC_Prep") \
    .config("spark.executor.memory", "3g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.network.timeout", "800s") \
    .config("spark.rpc.askTimeout", "800s") \
    .getOrCreate()

# --- 2. Définition des Paramètres Géographiques et HDFS ---
# Boîte englobante de la région de NYC
MIN_LAT, MAX_LAT = 40.0, 41.5
MIN_LON, MAX_LON = -75.0, -73.0

# Chemin HDFS BRUT (Corrigé avec l'hôte et le port du namenode)
RAW_OUTPUT_PATH = "hdfs://namenode:9000/user/mathis/datalake/noaa_gsod_nyc_raw_2005_2023.parquet"

# Plage d'années
START_YEAR = 2005
END_YEAR = 2023

print("Session Spark initialisée avec les configurations de robustesse.")

Session Spark initialisée avec les configurations de robustesse.


Phase 2 : Métadonnées et Identification des Stations NOAA

In [4]:
# --- 1. Téléchargement des Métadonnées des Stations (via Pandas car petit fichier) ---
stations_url = "https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv"
pdf_stations = pd.read_csv(stations_url,
                         dtype={'USAF': str, 'WBAN': str})

pdf_stations['STN_ID'] = pdf_stations['USAF'].str.strip() + pdf_stations['WBAN'].str.strip()
pdf_stations = pdf_stations.rename(columns={'LAT': 'LATITUDE', 'LON': 'LONGITUDE'})
pdf_stations = pdf_stations.dropna(subset=['LATITUDE', 'LONGITUDE', 'STATION NAME'])
spark_stations_df = spark.createDataFrame(pdf_stations)

# --- 2. Filtrage Géographique ---
nyc_stations_spark = spark_stations_df.filter(
    (F.col('LATITUDE') >= MIN_LAT) & (F.col('LATITUDE') <= MAX_LAT) &
    (F.col('LONGITUDE') >= MIN_LON) & (F.col('LONGITUDE') <= MAX_LON)
)

# Récupération de la liste des IDs pertinents (pour filtrage par nom de fichier)
relevant_station_ids = [row.STN_ID for row in nyc_stations_spark.select("STN_ID").collect()]

print(f"\n✅ {nyc_stations_spark.count()} Stations NOAA pertinentes trouvées près de New York.")
# Gardons ce DataFrame pour la jointure des coordonnées plus tard


✅ 93 Stations NOAA pertinentes trouvées près de New York.


Phase 2 bis : Téléchargement des données

In [7]:
import os
import requests
import time
from tqdm import tqdm # Importation de tqdm pour une barre de progression

# --- Paramètres de Configuration ---
BASE_URL = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access"
LOCAL_BASE_DIR = "/home/jovyan/work/data/noaa_gsod"
START_YEAR = 2005
END_YEAR = 2023

# --- IDs des stations à télécharger ---
# NOTE: Cette liste doit être remplie avec le résultat de votre Phase 2 !
# Exemple de données de test si la Phase 2 n'est pas exécutée :
# relevant_station_ids = ["72503094728", "72503014732"] 

# Si vous exécutez ce bloc après la Phase 2, assurez-vous que la liste est disponible.
if 'relevant_station_ids' not in locals():
    print("⚠️ ATTENTION: La liste 'relevant_station_ids' n'est pas définie. Veuillez exécuter la Phase 2 en premier.")
    # On sort du script si la liste n'est pas disponible pour éviter de télécharger inutilement
    exit()

# Démarrage du processus de téléchargement
print(f"Démarrage du téléchargement pour {len(relevant_station_ids)} stations de {START_YEAR} à {END_YEAR}.")

total_files = len(relevant_station_ids) * (END_YEAR - START_YEAR + 1)
downloaded_count = 0

# Utilisation de tqdm pour la barre de progression
with tqdm(total=total_files, desc="Téléchargement des fichiers GSOD") as pbar:
    
    for year in range(START_YEAR, END_YEAR + 1):
        year_dir = os.path.join(LOCAL_BASE_DIR, str(year))
        
        # Crée le répertoire de l'année s'il n'existe pas
        os.makedirs(year_dir, exist_ok=True)
        
        for station_id in relevant_station_ids:
            file_name = f"{station_id}.csv"
            local_path = os.path.join(year_dir, file_name)
            remote_url = f"{BASE_URL}/{year}/{file_name}"

            # Vérifie si le fichier existe déjà pour éviter de le re-télécharger
            if os.path.exists(local_path):
                # print(f"Fichier existant: {local_path}. Ignoré.")
                pbar.update(1)
                downloaded_count += 1
                continue
            
            try:
                # Requête HTTP GET pour télécharger le fichier
                response = requests.get(remote_url, timeout=10)
                response.raise_for_status() # Lève une exception si le statut HTTP est 4xx ou 5xx
                
                # Écrit le contenu dans le fichier local
                with open(local_path, 'wb') as f:
                    f.write(response.content)
                
                downloaded_count += 1
                pbar.update(1)
                # Pause pour être poli avec le serveur NOAA
                time.sleep(0.1) 
                
            except requests.exceptions.HTTPError as errh:
                # Fichier 404/Not Found (la station n'a pas forcément de données pour cette année)
                if response.status_code == 404:
                    pass # On ignore simplement ce fichier manquant
                else:
                    print(f"\n❌ Erreur HTTP pour {remote_url}: {errh}")
            except requests.exceptions.RequestException as e:
                print(f"\n❌ Erreur de Connexion/Timeout pour {remote_url}: {e}")

print(f"\n✅ Téléchargement terminé. {downloaded_count} fichiers GSOD traités (téléchargés ou existants).")

Démarrage du téléchargement pour 93 stations de 2005 à 2023.


Téléchargement des fichiers GSOD:  31%|███       | 552/1767 [22:41<49:56,  2.47s/it]  


✅ Téléchargement terminé. 552 fichiers GSOD traités (téléchargés ou existants).


Phase 3 : Ingestion Ciblée et Persistance (Couche Raw)

In [19]:
# --- Configuration des Chemins ---
LOCAL_BASE_DIR = "/home/jovyan/work/data/noaa_gsod" 
RAW_OUTPUT_PATH = "hdfs://namenode:9000/user/mathis/datalake/noaa_gsod_nyc_raw_2005_2023.parquet"
START_YEAR = 2005
END_YEAR = 2023

# --- 1. Définition des Chemins Ciblés ---
# Nous recréons la liste, mais cette fois en utilisant 'glob' ou une vérification OS
# pour ne pas inclure les chemins qui n'existent pas.

existing_targeted_paths = []
for year in range(START_YEAR, END_YEAR + 1):
    for station_id in relevant_station_ids:
        # Chemin absolu corrigé : /home/jovyan/work/data/noaa_gsod/2005/XXXXX.csv
        path = f"{LOCAL_BASE_DIR}/{year}/{station_id}.csv"
        
        # Vérifie si le fichier existe vraiment avant de l'ajouter à la liste de lecture de Spark
        if os.path.exists(path):
            existing_targeted_paths.append(path)

# Si aucun chemin n'existe, nous aurons une erreur, mais au moins nous savons pourquoi.
if not existing_targeted_paths:
    raise FileNotFoundError("Aucun fichier GSOD cible n'a été trouvé dans le répertoire local.")

gsod_data_paths = existing_targeted_paths
print(f"Total de {len(gsod_data_paths)} fichiers existants seront lus par Spark.")

# --- 2. Schéma et Lecture ---
gsod_schema = StructType([
    StructField("STATION", StringType(), True),
    StructField("DATE", StringType(), True),
    StructField("LATITUDE", DoubleType(), True), 
    StructField("LONGITUDE", DoubleType(), True),
    StructField("ELEVATION", DoubleType(), True),
    StructField("NAME", StringType(), True),
    StructField("TEMP", DoubleType(), True),
    StructField("TEMP_ATTRIBUTES", StringType(), True),
    StructField("DEWP", DoubleType(), True),
    StructField("DEWP_ATTRIBUTES", StringType(), True),
    StructField("SLP", DoubleType(), True),
    StructField("SLP_ATTRIBUTES", StringType(), True),
    StructField("STP", DoubleType(), True),
    StructField("STP_ATTRIBUTES", StringType(), True),
    StructField("VISIB", DoubleType(), True),
    StructField("VISIB_ATTRIBUTES", StringType(), True),
    StructField("WDSP", DoubleType(), True),
    StructField("WDSP_ATTRIBUTES", StringType(), True),
    StructField("MXSPD", DoubleType(), True),
    StructField("GUST", DoubleType(), True),
    StructField("MAX", DoubleType(), True),
    StructField("MAX_ATTRIBUTES", StringType(), True),
    StructField("MIN", DoubleType(), True),
    StructField("MIN_ATTRIBUTES", StringType(), True),
    StructField("PRCP", DoubleType(), True),
    StructField("PRCP_ATTRIBUTES", StringType(), True),
    StructField("SNDP", DoubleType(), True),
    StructField("FRSHHT", StringType(), True),
])

# Lecture distribuée des données GSOD (seulement les fichiers ciblés)
all_gsod_data = spark.read.csv(
    gsod_data_paths,
    header=True,
    schema=gsod_schema,
    sep=','
)

# Renommage de la colonne ID
nyc_gsod_data = all_gsod_data.withColumnRenamed("STATION", "ID_STATION")


# --- 3. Persistance de la Couche Brute sur HDFS ---
print(f"\nSauvegarde de la copie BRUTE filtrée (2005-2023) dans : {RAW_OUTPUT_PATH}...")
# Cette étape transfère les données du disque local du conteneur vers HDFS
nyc_gsod_data.write.mode("overwrite").parquet(RAW_OUTPUT_PATH)
print("✅ Copie brute sauvegardée sur HDFS. Le traitement peut se poursuivre.")

Total de 552 fichiers existants seront lus par Spark.

Sauvegarde de la copie BRUTE filtrée (2005-2023) dans : hdfs://namenode:9000/user/mathis/datalake/noaa_gsod_nyc_raw_2005_2023.parquet...
✅ Copie brute sauvegardée sur HDFS. Le traitement peut se poursuivre.


Phase 4 : Nettoyage et Jointure des Coordonnées (ETL)

In [20]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

# --- Chemins de configuration ---
RAW_OUTPUT_PATH = "hdfs://namenode:9000/user/mathis/datalake/noaa_gsod_nyc_raw_2005_2023.parquet"
# nyc_stations_spark doit être le DataFrame des stations filtrées de la Phase 2

# Lecture du fichier Parquet depuis HDFS pour la couche de Traitement/Nettoyage (ETL)
data_for_cleaning = spark.read.parquet(RAW_OUTPUT_PATH)

# --- 1. Nettoyage et Renommage des Colonnes ---
renaming_map = {
    "DATE": "DATE_OBSERVATION",
    "TEMP": "TEMP_MOYENNE_F",
    "DEWP": "POINT_ROSEE_F",
    "PRCP": "PRECIPITATION_POUCES",
    "MIN": "TEMP_MIN_F",
    "MAX": "TEMP_MAX_F",
    "WDSP": "VITESSE_VENT_NOEUDS",
    "VISIB": "VISIBILITE_MILLES",
    "SLP": "PRESSION_ATM_MER",
    "FRSHHT": "PHENOMENES"
}

gsod_renamed_df = data_for_cleaning
for old_name, new_name in renaming_map.items():
    if old_name in gsod_renamed_df.columns:
        gsod_renamed_df = gsod_renamed_df.withColumnRenamed(old_name, new_name)


# --- 2. Jointure des Coordonnées des Stations ---
# Ajoute la LATITUDE_STATION et LONGITUDE_STATION pour le calcul de distance
clean_final_gsod_df = gsod_renamed_df.join(
    nyc_stations_spark.select(
        'STN_ID', 
        F.col('LATITUDE').alias('LATITUDE_STATION'), 
        F.col('LONGITUDE').alias('LONGITUDE_STATION')
    ), 
    on=[F.col("ID_STATION") == F.col("STN_ID")],
    how='left'
).drop("STN_ID").drop("LATITUDE").drop("LONGITUDE") # Supprime les colonnes brutes des coordonnées


print("\nDonnées NOAA nettoyées et enrichies des coordonnées des stations (Couche ETL Complète).")
clean_final_gsod_df.printSchema()


Données NOAA nettoyées et enrichies des coordonnées des stations (Couche ETL Complète).
root
 |-- ID_STATION: string (nullable = true)
 |-- DATE_OBSERVATION: string (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- NAME: string (nullable = true)
 |-- TEMP_MOYENNE_F: double (nullable = true)
 |-- TEMP_ATTRIBUTES: string (nullable = true)
 |-- POINT_ROSEE_F: double (nullable = true)
 |-- DEWP_ATTRIBUTES: string (nullable = true)
 |-- PRESSION_ATM_MER: double (nullable = true)
 |-- SLP_ATTRIBUTES: string (nullable = true)
 |-- STP: double (nullable = true)
 |-- STP_ATTRIBUTES: string (nullable = true)
 |-- VISIBILITE_MILLES: double (nullable = true)
 |-- VISIB_ATTRIBUTES: string (nullable = true)
 |-- VITESSE_VENT_NOEUDS: double (nullable = true)
 |-- WDSP_ATTRIBUTES: string (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- TEMP_MAX_F: double (nullable = true)
 |-- MAX_ATTRIBUTES: string (nullable = true)
 |-- TEMP_MIN_F: double

Phase 5 : Préparation des Coordonnées des Zones de Qualité de l'Air

In [21]:
# --- 1. Téléchargement et Extraction du GeoJSON ---
geojson_url = "https://raw.githubusercontent.com/nycehs/NYC_geography/master/UHF42.geo.json"

response = requests.get(geojson_url)
geo_data_raw = response.json()

geo_records = []
for feature in geo_data_raw['features']:
    properties = feature['properties']
    geometry = feature['geometry']
    coords = geometry['coordinates']
    
    try:
        # Approximation du centroïde pour la distance
        if geometry['type'] == 'Polygon':
            lon = coords[0][0][0]
            lat = coords[0][0][1]
        elif geometry['type'] == 'MultiPolygon':
            lon = coords[0][0][0][0]
            lat = coords[0][0][0][1]
        else:
            continue
            
        geo_records.append({
            "GEOJOIN_ID": properties['GEOCODE'], 
            "LATITUDE_ZONE": lat,
            "LONGITUDE_ZONE": lon
        })
    except (IndexError, TypeError):
        continue

# --- 2. Création du DataFrame Spark des Coordonnées des Zones ---
geo_schema = StructType([
    StructField("GEOJOIN_ID", StringType(), False),
    StructField("LATITUDE_ZONE", DoubleType(), True),
    StructField("LONGITUDE_ZONE", DoubleType(), True)
])

geo_df = spark.createDataFrame(geo_records, schema=geo_schema)

print(f"\n{geo_df.count()} zones de qualité de l'air (GeoJoin ID) préparées avec leurs coordonnées.")


43 zones de qualité de l'air (GeoJoin ID) préparées avec leurs coordonnées.


Phase 6 : Calcul de la Distance (Haversine) et Voisin le Plus Proche

In [22]:
from pyspark.sql.window import Window

# Constante du rayon de la Terre en kilomètres (km)
R = 6371.0

# Formule de la Haversine (UDF)
def haversine(lon1, lat1, lon2, lat2):
    # Conversion de degrés en radians
    lon1_rad, lat1_rad, lon2_rad, lat2_rad = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    
    return R * c

# Enregistrement de la fonction comme une UDF (User Defined Function) pour Spark
haversine_udf = F.udf(haversine, DoubleType())

# --- 1. Préparation des Coordonnées des Stations Uniques ---
stations_coords_df = clean_final_gsod_df.select(
    "ID_STATION", "LATITUDE_STATION", "LONGITUDE_STATION"
).distinct()

# --- 2. Jointure Cartésienne (Toutes les zones vs. Toutes les stations) ---
cross_joined_df = geo_df.crossJoin(stations_coords_df)


# --- 3. Calcul de la Distance pour chaque paire ---
distance_df = cross_joined_df.withColumn(
    "DISTANCE_KM",
    haversine_udf(
        F.col("LONGITUDE_ZONE"), 
        F.col("LATITUDE_ZONE"), 
        F.col("LONGITUDE_STATION"), 
        F.col("LATITUDE_STATION")
    )
)

# --- 4. Identification du Voisin le Plus Proche ---
# Trouve la ligne (station) avec la distance minimale pour chaque GEOJOIN_ID
window_spec = Window.partitionBy("GEOJOIN_ID").orderBy(F.col("DISTANCE_KM"))

nearest_station_df = distance_df.withColumn(
    "rank", 
    F.rank().over(window_spec)
).filter(F.col("rank") == 1).drop("rank")


print("\n✅ Table de correspondance (Voisin le plus proche) créée :")
nearest_station_df.select(
    "GEOJOIN_ID", 
    "ID_STATION", 
    "DISTANCE_KM"
).show(5, truncate=False)


✅ Table de correspondance (Voisin le plus proche) créée :
+----------+-----------+------------------+
|GEOJOIN_ID|ID_STATION |DISTANCE_KM       |
+----------+-----------+------------------+
|0         |74486094789|5.760247595051896 |
|101       |72503014732|14.074131616303024|
|102       |99728099999|9.640847117034312 |
|103       |99728099999|11.103908014210129|
|104       |72503014732|4.739471712784995 |
+----------+-----------+------------------+
only showing top 5 rows



Phase 7 : Jointure Finale et Persistance (Couche Insight)

In [23]:
import pyspark.sql.functions as F

# --- Configuration HDFS ---
# Chemin où sera stocké le jeu de données final, prêt pour l'analyse
INSIGHT_OUTPUT_PATH = "hdfs://namenode:9000/user/mathis/datalake/noaa_gsod_nyc_enriched_clean.parquet"

# --- 1. Jointure d'Enrichissement ---
# On joint toutes les observations météo (clean_final_gsod_df) aux informations de GeoJoin ID (nearest_station_df)
# La jointure s'effectue sur l'ID de la station météo.

# NOTE: Ce bloc nécessite d'avoir exécuté la Phase 6 juste avant pour que 'nearest_station_df'
# et 'clean_final_gsod_df' soient disponibles en mémoire.

final_insights_df = clean_final_gsod_df.join(
    nearest_station_df.select("GEOJOIN_ID", 
                              F.col("ID_STATION").alias("NEAREST_STATION_ID"), 
                              "DISTANCE_KM",
                              "LATITUDE_ZONE", 
                              "LONGITUDE_ZONE"),
    # La condition est que l'ID de la station météo DOIT correspondre à l'ID du voisin le plus proche.
    F.col("ID_STATION") == F.col("NEAREST_STATION_ID"),
    "inner" 
).drop("NEAREST_STATION_ID") # On retire cette colonne après la jointure


# --- 2. Sélection et Ordre Final ---
# Crée le schéma final pour l'analyse, en plaçant les clés d'analyse en tête
final_insights_df = final_insights_df.select(
    "GEOJOIN_ID", 
    "DATE_OBSERVATION", 
    # Mesures Météo
    "TEMP_MOYENNE_F", 
    "PRECIPITATION_POUCES", 
    "VITESSE_VENT_NOEUDS", 
    "PHENOMENES",
    # Méta-données de jointure
    "ID_STATION", 
    "DISTANCE_KM",
    "LATITUDE_ZONE", 
    "LONGITUDE_ZONE" # Coordonnées de la zone (utile pour la cartographie)
)

print("\nStructure du DataFrame Final (Météo attribuée à chaque zone de qualité de l'air) :")
final_insights_df.printSchema()
final_insights_df.show(5)

# --- 3. Persistance de la Couche Insight sur HDFS ---
print(f"\nSauvegarde du DataFrame final Nettoyé/Enrichi (Couche Insight) dans : {INSIGHT_OUTPUT_PATH}...")
# Sauvegarde au format Parquet
final_insights_df.write.mode("overwrite").parquet(INSIGHT_OUTPUT_PATH)
print("✅ Jointure finale et persistance terminées. Le jeu de données météo est prêt pour l'analyse.")


Structure du DataFrame Final (Météo attribuée à chaque zone de qualité de l'air) :
root
 |-- GEOJOIN_ID: string (nullable = false)
 |-- DATE_OBSERVATION: string (nullable = true)
 |-- TEMP_MOYENNE_F: double (nullable = true)
 |-- PRECIPITATION_POUCES: double (nullable = true)
 |-- VITESSE_VENT_NOEUDS: double (nullable = true)
 |-- PHENOMENES: string (nullable = true)
 |-- ID_STATION: string (nullable = true)
 |-- DISTANCE_KM: double (nullable = true)
 |-- LATITUDE_ZONE: double (nullable = true)
 |-- LONGITUDE_ZONE: double (nullable = true)

+----------+----------------+--------------+--------------------+-------------------+----------+-----------+------------------+-------------+--------------+
|GEOJOIN_ID|DATE_OBSERVATION|TEMP_MOYENNE_F|PRECIPITATION_POUCES|VITESSE_VENT_NOEUDS|PHENOMENES| ID_STATION|       DISTANCE_KM|LATITUDE_ZONE|LONGITUDE_ZONE|
+----------+----------------+--------------+--------------------+-------------------+----------+-----------+------------------+-----------

Phase 8 (A) : Téléchargement et Nettoyage du JSON Socrata

In [9]:
import requests
import json
import os

# --- Configuration des Chemins ---
LOCAL_BASE_DIR = "/home/jovyan/work/data/air_quality"
LOCAL_JSON_PATH = os.path.join(LOCAL_BASE_DIR, "nyc_air_quality_raw.json")
AIR_QUALITY_URL = "https://data.cityofnewyork.us/api/views/c3uy-2p5r/rows.json?accessType=DOWNLOAD"

# Crée le répertoire local si nécessaire
os.makedirs(LOCAL_BASE_DIR, exist_ok=True)

# --- 1. Téléchargement et Nettoyage de la structure JSON Socrata ---
print(f"⬇️ Téléchargement du JSON Socrata depuis l'API de NYC...")
try:
    response = requests.get(AIR_QUALITY_URL, timeout=300) # Timeout de 5 minutes
    response.raise_for_status()
    data = response.json()
    
    # La clé 'data' contient le tableau des enregistrements bruts que Spark doit lire.
    raw_records = data.get('data', [])

    if not raw_records:
        print("❌ Erreur : La clé 'data' est vide dans le JSON téléchargé. Arrêt du processus.")
        exit()
    
    # Écriture du tableau de données brutes SEULEMENT dans le nouveau fichier JSON.
    # Ceci est essentiel pour que le RDD/toDF fonctionne correctement.
    with open(LOCAL_JSON_PATH, 'w') as f:
        json.dump(raw_records, f)

    print(f"✅ Fichier JSON brut sauvegardé et nettoyé structurellement à : {LOCAL_JSON_PATH}")
    
except Exception as e:
    print(f"❌ Erreur lors du téléchargement/nettoyage : {e}")
    exit()

⬇️ Téléchargement du JSON Socrata depuis l'API de NYC...
✅ Fichier JSON brut sauvegardé et nettoyé structurellement à : /home/jovyan/work/data/air_quality/nyc_air_quality_raw.json


Phase 8 (B) : Ingestion, Normalisation et Pivotage de la Qualité de l'Air

In [11]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import requests
import json
import os

# --- Configurations des Chemins ---
LOCAL_BASE_DIR = "/home/jovyan/work/data/air_quality"
LOCAL_JSON_PATH = os.path.join(LOCAL_BASE_DIR, "nyc_air_quality_raw.json")
AIR_QUALITY_URL = "https://data.cityofnewyork.us/api/views/c3uy-2p5r/rows.json?accessType=DOWNLOAD"
AIR_QUALITY_OUTPUT_PATH = "hdfs://namenode:9000/user/mathis/datalake/nyc_air_quality_pivot.parquet"

# Crée le répertoire local si nécessaire
os.makedirs(LOCAL_BASE_DIR, exist_ok=True)


# --- 1. Lecture du tableau JSON en mémoire Python (Assumée réussie) ---
# Ce bloc dépend du succès de l'étape de téléchargement et de nettoyage du JSON brut.
with open(LOCAL_JSON_PATH, 'r') as f:
    raw_socrata_array = json.load(f)

# --- 2. Définition du Schéma RDD/toDF : Correction du format Socrata ---

# Mappage des index Socrata confirmés (Index dans le tableau data)
column_names = {
    17: "DATE_MESURE_BRUTE",         # Index 17 -> Start_Date
    14: "GEOJOIN_ID_BRUT",           # Index 14 -> Geo Join ID
    10: "NOM_POLLUANT",              # Index 10 -> Name (Polluant)
    18: "VALEUR_MESURE_BRUTE"        # Index 18 -> Data Value
}

# Définir tous les noms de colonnes jusqu'à l'indice max (19)
# La correction est ici : range(20) pour obtenir 20 colonnes (0 à 19)
col_labels = [f"col_{i}" for i in range(20)] # ANCIEN: range(19)

# Mappage des indices Socrata confirmés (inchangé, ils sont tous < 20)
column_names = {
    17: "DATE_MESURE_BRUTE",
    14: "GEOJOIN_ID_BRUT",
    10: "NOM_POLLUANT",
    18: "VALEUR_MESURE_BRUTE"
}

# (Le reste du mappage est inchangé)
for index, name in column_names.items():
    col_labels[index] = name

# CRÉATION DU SCHÉMA STRUCTURÉ (Tous en StringType)
target_schema = StructType([
    StructField(name, StringType(), True) for name in col_labels
])

# Création du DataFrame Spark à partir du RDD avec le schéma explicite
air_quality_data_raw = spark.sparkContext.parallelize(raw_socrata_array).toDF(target_schema)

print(f"Qualité de l'air : {air_quality_data_raw.count()} enregistrements bruts lus.")
air_quality_data_raw.printSchema()


# --- 3. Sélection et Nettoyage des Colonnes (ETL) ---

air_quality_df = air_quality_data_raw.select(
    "DATE_MESURE_BRUTE",
    "GEOJOIN_ID_BRUT",
    "NOM_POLLUANT",
    "VALEUR_MESURE_BRUTE"
)

air_quality_clean = air_quality_df.filter(
    F.col("VALEUR_MESURE_BRUTE").isNotNull()
).withColumn(
    # Conversion de la valeur de mesure en Double
    "VALEUR_MESURE", F.col("VALEUR_MESURE_BRUTE").cast(DoubleType())
).withColumn(
    # GeoJoin ID en String pour la jointure
    "GEOJOIN_ID", F.col("GEOJOIN_ID_BRUT").cast(StringType())
).withColumn(
    # Conversion de la date (YYYY-MM-DD...) en format YYYYMMDD entier pour la jointure avec NOAA.
    "DATE_OBSERVATION", 
    F.regexp_replace(F.substring(F.col("DATE_MESURE_BRUTE"), 1, 10), "-", "").cast(IntegerType())
).drop("DATE_MESURE_BRUTE", "VALEUR_MESURE_BRUTE", "GEOJOIN_ID_BRUT")

# --- 4. Croisement (Pivot) des Données de Qualité de l'Air ---
# Transforme les lignes de polluants en colonnes
air_quality_pivot = air_quality_clean.groupBy("GEOJOIN_ID", "DATE_OBSERVATION").pivot("NOM_POLLUANT").agg(
    F.mean("VALEUR_MESURE")
)

# Renommage des colonnes pivotées pour la clarté
for col_name in air_quality_pivot.columns:
    if col_name not in ["GEOJOIN_ID", "DATE_OBSERVATION"]:
        # Nettoyage du nom du polluant
        new_col_name = f"QA_{col_name.replace(' ', '_').replace('.', '_').replace('(', '').replace(')', '').replace('/', '_')}_MOYENNE"
        air_quality_pivot = air_quality_pivot.withColumnRenamed(col_name, new_col_name)

print("\nStructure pivotée des données de qualité de l'air (prête pour la jointure finale) :")
air_quality_pivot.printSchema()
air_quality_pivot.show(5)

# --- 5. Persistance sur HDFS ---
air_quality_pivot.write.mode("overwrite").parquet(AIR_QUALITY_OUTPUT_PATH)
print(f"✅ Qualité de l'air pivotée et persistée sur HDFS à : {AIR_QUALITY_OUTPUT_PATH}")

Qualité de l'air : 18862 enregistrements bruts lus.
root
 |-- col_0: string (nullable = true)
 |-- col_1: string (nullable = true)
 |-- col_2: string (nullable = true)
 |-- col_3: string (nullable = true)
 |-- col_4: string (nullable = true)
 |-- col_5: string (nullable = true)
 |-- col_6: string (nullable = true)
 |-- col_7: string (nullable = true)
 |-- col_8: string (nullable = true)
 |-- col_9: string (nullable = true)
 |-- NOM_POLLUANT: string (nullable = true)
 |-- col_11: string (nullable = true)
 |-- col_12: string (nullable = true)
 |-- col_13: string (nullable = true)
 |-- GEOJOIN_ID_BRUT: string (nullable = true)
 |-- col_15: string (nullable = true)
 |-- col_16: string (nullable = true)
 |-- DATE_MESURE_BRUTE: string (nullable = true)
 |-- VALEUR_MESURE_BRUTE: string (nullable = true)
 |-- col_19: string (nullable = true)


Structure pivotée des données de qualité de l'air (prête pour la jointure finale) :
root
 |-- GEOJOIN_ID: string (nullable = true)
 |-- DATE_OBSERVATION

Phase 9 : Jointure Finale et Persistance du Jeu de Données Complet

In [12]:
import pyspark.sql.functions as F

# --- Configuration des Chemins (Réutilisés) ---
# Chemin des données NOAA enrichies (Couche Insight après Phase 7)
INSIGHT_OUTPUT_PATH = "hdfs://namenode:9000/user/mathis/datalake/noaa_gsod_nyc_enriched_clean.parquet"
# Chemin des données Qualité de l'Air pivotées (Couche Pivot après Phase 8)
AIR_QUALITY_OUTPUT_PATH = "hdfs://namenode:9000/user/mathis/datalake/nyc_air_quality_pivot.parquet"
# Chemin du jeu de données final
FINAL_DATASET_PATH = "hdfs://namenode:9000/user/mathis/datalake/nyc_final_air_weather_dataset.parquet"

# --- 1. Lecture des deux sources enrichies depuis HDFS ---
final_insights_df = spark.read.parquet(INSIGHT_OUTPUT_PATH)
air_quality_pivot = spark.read.parquet(AIR_QUALITY_OUTPUT_PATH)

# --- 2. Jointure Finale Temporelle et Géographique ---
# La jointure se fait sur deux clés : l'ID de la zone et la Date de l'observation
final_dataset = final_insights_df.join(
    air_quality_pivot,
    on=["GEOJOIN_ID", "DATE_OBSERVATION"],
    how="inner" # Utilisation de 'inner' pour ne garder que les jours où les DEUX mesures sont présentes
)

# --- 3. Sélection Finale et Sauvegarde ---
# Réorganisation et sélection des colonnes pour le jeu de données d'analyse finale
final_dataset = final_dataset.select(
    "GEOJOIN_ID", 
    "DATE_OBSERVATION", 
    # Météo
    "TEMP_MOYENNE_F", 
    "PRECIPITATION_POUCES", 
    "VITESSE_VENT_NOEUDS", 
    "PHENOMENES",
    # Qualité de l'Air (toutes les colonnes commençant par QA_)
    *[col for col in final_dataset.columns if col.startswith("QA_")],
    # Métadonnées de jointure
    F.col("DISTANCE_KM").alias("DISTANCE_STATION_KM"),
    "ID_STATION",
    "LATITUDE_ZONE", 
    "LONGITUDE_ZONE" 
)

print("\nStructure du Jeu de Données Final (Météo + Qualité de l'Air) :")
final_dataset.printSchema()
final_dataset.show(5)

# --- 4. Persistance Finale sur HDFS ---
final_dataset.write.mode("overwrite").parquet(FINAL_DATASET_PATH)

print(f"\n✅ Jeu de données final (Météo + Qualité de l'Air) créé et persisté à : {FINAL_DATASET_PATH}")
print("Le pipeline d'Ingestion, Persistance et Traitement est terminé. Votre jeu de données est prêt pour l'Insight. 📊")


Structure du Jeu de Données Final (Météo + Qualité de l'Air) :
root
 |-- GEOJOIN_ID: string (nullable = true)
 |-- DATE_OBSERVATION: string (nullable = true)
 |-- TEMP_MOYENNE_F: double (nullable = true)
 |-- PRECIPITATION_POUCES: double (nullable = true)
 |-- VITESSE_VENT_NOEUDS: double (nullable = true)
 |-- PHENOMENES: string (nullable = true)
 |-- QA_Annual_vehicle_miles_traveled_MOYENNE: double (nullable = true)
 |-- QA_Annual_vehicle_miles_traveled_cars_MOYENNE: double (nullable = true)
 |-- QA_Annual_vehicle_miles_traveled_trucks_MOYENNE: double (nullable = true)
 |-- QA_Asthma_emergency_department_visits_due_to_PM2_5_MOYENNE: double (nullable = true)
 |-- QA_Asthma_emergency_departments_visits_due_to_Ozone_MOYENNE: double (nullable = true)
 |-- QA_Asthma_hospitalizations_due_to_Ozone_MOYENNE: double (nullable = true)
 |-- QA_Boiler_Emissions-_Total_NOx_Emissions_MOYENNE: double (nullable = true)
 |-- QA_Boiler_Emissions-_Total_PM2_5_Emissions_MOYENNE: double (nullable = true)
